In [1]:
import nltk

nltk.download('treebank')

!pip install pytorch-pretrained-bert
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel

from nltk.corpus import treebank_chunk
import pandas as pd

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
    100% |████████████████████████████████| 122kB 4.4MB/s 
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
import torch.nn as nn
import torch

In [3]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpqbiubjkw
100%|██████████| 231508/231508 [00:00<00:00, 2629054.34B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpqbiubjkw to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpqbiubjkw
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f0006

In [4]:
train_data = [r for r in treebank_chunk.tagged_sents()]
train_sent = []
for s in train_data:
  train_sent.append(" ".join([t[0] for t in s]))
train_sent = ["[CLS] "+i+" [SEP]" for i in train_sent]

/usr/local/lib/python3.6/dist-packages/nltk/tokenize/regexp.py:123: FutureWarning: split() requires a non-empty pattern match.
  return [tok for tok in self._regexp.split(text) if tok]


In [0]:
tokenized_text = [tokenizer.tokenize(s) for s in train_sent]

In [0]:
# Convert token to vocabulary indices
indexed_tokens = [tokenizer.convert_tokens_to_ids(s) for s in tokenized_text]

In [0]:
labels_pos = []
for s,d in zip(tokenized_text,train_data):
  temp = []
  t_list = d.copy()
  for t in s:
    
    if t == '[CLS]' :
      temp.append("[START]")
      continue
    if t == '[SEP]':
      temp.append("[STOP]")
      continue      
    if t.startswith("##"):
      temp.append("O")
      continue
    if len(t_list)!=0:
      if t_list[0][0].lower().startswith(t):
        temp.append(t_list[0][1])
        t_list.pop(0)
        continue
    temp.append("O")
    continue

    
  labels_pos.append(temp)

  
  

In [0]:
label_set = []
for l in labels_pos:
  label_set.extend(l)
label_set = set(label_set)

In [0]:
idx2pos = {i:j for i,j in enumerate(label_set)}

pos2idx = {j:i for i,j in idx2pos.items()}

idx2pos[pos2idx['O']] = idx2pos[0]
idx2pos[0] = 'O'

pos2idx = {j:i for i,j in idx2pos.items()}

In [10]:
pos2idx

{'#': 5,
 '$': 2,
 "''": 42,
 '(': 28,
 ')': 9,
 ',': 30,
 '.': 40,
 ':': 20,
 'CC': 45,
 'CD': 10,
 'DT': 31,
 'EX': 15,
 'FW': 34,
 'IN': 22,
 'JJ': 32,
 'JJR': 33,
 'JJS': 25,
 'LS': 11,
 'MD': 29,
 'NN': 35,
 'NNP': 36,
 'NNPS': 41,
 'NNS': 16,
 'O': 0,
 'PDT': 6,
 'POS': 26,
 'PRP': 38,
 'PRP$': 24,
 'RB': 14,
 'RBR': 46,
 'RBS': 48,
 'RP': 4,
 'SYM': 27,
 'TO': 1,
 'UH': 18,
 'VB': 19,
 'VBD': 37,
 'VBG': 12,
 'VBG|NN': 43,
 'VBN': 13,
 'VBP': 44,
 'VBZ': 47,
 'WDT': 39,
 'WP': 8,
 'WP$': 21,
 'WRB': 23,
 '[START]': 3,
 '[STOP]': 17,
 '``': 7}

In [0]:
labels_idx = []
for s in labels_pos:
  temp = []
  for p in s:
    temp.append(pos2idx[p])
  labels_idx.append(temp)

In [0]:
train_data,valid_data = train_data[0:4000],train_data[4000:4009]
train_tokenized_text,valid_tokenized_text = tokenized_text[0:4000],tokenized_text[4000:4009]
train_indexed_tokens,valid_indexed_tokens = indexed_tokens[0:4000],indexed_tokens[4000:4009]
train_labels_pos,valid_labels_pos = labels_pos[0:4000],labels_pos[4000:4009]
train_labels_idx,valid_labels_idx = labels_idx[0:4000],labels_idx[4000:4009]

In [94]:
from torch.nn.utils.rnn import pad_sequence
sub_batch = 4
acc_time = 4
acc_counter = 0
EPOCHS = 10



loss_fn = nn.CrossEntropyLoss(ignore_index = 0)#weight = torch.Tensor([1.0,1.0,5.0]))
loss_fn.cuda()
bert = BertModel.from_pretrained('bert-base-uncased')
bert =  bert.cuda()
bert.train()
linear = nn.Linear(768,49).cuda()

opt = torch.optim.Adam(list(bert.parameters()) + list(linear.parameters()), lr=2e-5)
for e in range(EPOCHS):
  for b in range(0,len(train_data),sub_batch):
    start = b
    end = min(len(train_data),b+sub_batch)

    tr_X = train_indexed_tokens[start:end]
    tr_Y = train_labels_idx[start:end]
    tr_X_padded = pad_sequence([torch.Tensor(v) for v in tr_X],batch_first = True).cuda().long()
    tr_Y_padded = pad_sequence([torch.Tensor(v) for v in tr_Y],batch_first = True).cuda().long()
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    attention_mask = (tr_X_padded != 0).cuda().long()
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    pos = linear(encoded_bert)

    #print (tr_Y_padded)
    
    pos = pos.view(-1,pos.size()[-1])
    
    tr_Y_padded = tr_Y_padded.view(-1)

    loss = loss_fn(pos,tr_Y_padded)

    loss.backward()
    acc_counter = acc_counter+1
    if acc_counter == acc_time:
      opt.step()
      acc_counter = 0
      opt.zero_grad()
    if b%100 == 0:
      print ("Epoch =",e,"loss = ",loss.item())

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp65f9g3ia
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Epoch = 0 loss =  4.024296760559082
Epoch = 0 loss =  3.548032283782959
Epoch = 0 loss =  3.1204402446746826
Epoch = 0 loss =  3.0281245708465576
Epoch = 0 loss =  2.545095443725586
Epoch = 0 loss =  2.4869539737701416
Epoch = 0 loss =  2.415323257446289
Epoch = 0 loss =  2.051082134246826
Epoch = 0 loss =  1.8237688541412354
Epoch = 0 loss =  1.6270627975463867
Epoch = 0 loss =  1.298710823059082
Epoch = 0 loss =  1.3668063879013062
Epoch = 0 loss =  1.041852355003357
Epoch = 0 loss =  0.848410964012146
Epoch = 0 loss =  0.6554121375083923
Epoch = 0 loss =  0.6439481973648071
Epoch = 0 loss =  0.7173029184341431
Epoch = 0 loss =  0.4560617506504059
Epoch = 0 loss =  0.5075008273124695
Epoch = 0 loss =  0.36558979749679565
Epoch = 0 loss =  0.36088627576828003
Epoch = 0 loss =  0.46057695150375366
Epoch = 0 loss =  0.6857959628105164
Epoch = 0 loss =  0.29613351821899414
Epoch = 0 loss =  0.230240598320961
Epoch = 0 loss =  0.20240087807178497
Epoch = 0 loss =  0.3518933057785034
Epoch

In [95]:
valid_trX = pad_sequence([torch.Tensor(v) for v in valid_indexed_tokens],batch_first = True).cuda().long()
valid_trY = pad_sequence([torch.Tensor(v) for v in valid_labels_idx],batch_first = True).cuda().long()

with torch.no_grad():
    bert.eval()
    segments_tensors = torch.zeros(valid_trX.size()).cuda().long()
    attention_mask = (valid_trX != 0).cuda().long()
    encoded_bert, _ = bert(valid_trX, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pos = linear(encoded_bert).view(-1,pos.shape[-1])
    valid_trY = valid_trY.view(-1)
    pred_trY = torch.argmax(pos,dim = -1)
    
print ("Acc = ",(valid_trY[valid_trY!=0] == pred_trY[valid_trY!=0]).float().mean())

Acc =  tensor(0.9839, device='cuda:0')


In [97]:
valid_trY = valid_trY.view(9,49)
pred_trY = pred_trY.view(9,49)
for i in range(9):
  for j in range(49):
    if valid_trY[i,j] != 0:
      if valid_trY[i,j] != pred_trY[i,j]:
        print (i,j)
        print (pred_trY[i,j],valid_trY[i,j])

3 9
tensor(37, device='cuda:0') tensor(13, device='cuda:0')
4 20
tensor(32, device='cuda:0') tensor(36, device='cuda:0')
4 21
tensor(35, device='cuda:0') tensor(36, device='cuda:0')
4 33
tensor(35, device='cuda:0') tensor(32, device='cuda:0')


In [52]:
pd.DataFrame(pred_trY.view(9,-1).cpu().numpy()).applymap(lambda x: idx2pos[x]).loc[0],pd.DataFrame(valid_trY.view(9,-1).cpu().numpy()).applymap(lambda x: idx2pos[x]).loc[0]

(0     [START]
 1          CC
 2          DT
 3          CD
 4          JJ
 5         NNS
 6           ,
 7         VBG
 8          TO
 9          DT
 10         NN
 11         IN
 12        NNP
 13          .
 14        NNP
 15        NNP
 16          (
 17        NNP
 18          .
 19          ,
 20        NNP
 21          .
 22          )
 23          ,
 24        VBD
 25         IN
 26         DT
 27         JJ
 28         NN
 29         IN
 30         DT
 31         NN
 32         ``
 33         ``
 34        VBZ
 35         DT
 36         JJ
 37         NN
 38         TO
 39         VB
 40         DT
 41         JJ
 42          :
 43         NN
 44         NN
 45          .
 46     [STOP]
 47         NN
 48         NN
 Name: 0, dtype: object, 0     [START]
 1          CC
 2          DT
 3          CD
 4          JJ
 5         NNS
 6           ,
 7         VBG
 8          TO
 9          DT
 10         NN
 11         IN
 12        NNP
 13          O
 14        NNP
 15        NNP
 

In [0]:
class CRF(nn.Module):
    def __init__(self,start_tag_idx,end_tag_idx,tags_num = 49,batch_size = 4):
        super(CRF,self).__init__()
        self.tags_num = tags_num
        self.transitions = nn.Parameter(torch.ones(tags_num,tags_num)) # 0 postion is for "O" so no one can use it
        self.start_tag_idx = start_tag_idx
        self.end_tag_idx = end_tag_idx
        self.batch_size = batch_size
        self.register_buffer('last_tag', torch.ones(self.batch_size).long())
        
           
        nn.init.uniform_(self.transitions, -0.01, 0.01)

        self.transitions.data[:, start_tag_idx] = -100000.0

        self.transitions.data[end_tag_idx, :] = -100000.0

    def score_sentence(self,batch_feat, batch_tag):
      
        self.batch_size, self.seq_length, self.hidden_dim = batch_feat.size()
        
        score = batch_feat[:,0,self.start_tag_idx].view(-1)
        
        nn.init.ones_(self.last_tag)
        
        self.last_tag = self.last_tag*self.start_tag_idx
        
        mask = (batch_tag != 0).float()

        for i in range(1,self.seq_length):

          score = score + batch_feat[:,i].gather(dim = 1,index = batch_tag[:,i].unsqueeze(1)).view(-1)*mask[:,i].view(-1)


          score = score + self.transitions[self.last_tag,batch_tag[:,i]].view(-1)*mask[:,i].view(-1)
          
          self.last_tag = torch.where(mask[:,i].byte().view(-1), batch_tag[:,i].view(-1), self.last_tag.view(-1))
                    
        return score
          
          
          
        
    def forward(self,batch_feat, batch_tag):
      
        score = self.score_sentence(batch_feat,batch_tag)
        
        partition = self.compute_log_partition(batch_feat,(batch_tag!=0))
        
        return partition - score
        
    def compute_log_partition(self,batch_feat,mask):
      
        mask = mask.float()
      
        self.batch_size, self.seq_length, self.hidden_dim = batch_feat.size()
        
        alphas = batch_feat[:, 0,:]
        
        for i in range(1, self.seq_length):
             
          
            alphas = alphas.unsqueeze(2)


            e_scores = batch_feat[:, i, :].unsqueeze(1)

            t_scores = self.transitions[:, :].unsqueeze(0)

            scores = e_scores + t_scores + alphas

            new_alphas = torch.logsumexp(scores, dim=1)
            
            alphas = alphas.squeeze()

            mask_t = mask[:, i].unsqueeze(-1)
            
            alphas = mask_t * new_alphas + (1 - mask_t) * alphas

        return torch.logsumexp(alphas, dim=1)
      
    def decode(self,batch_feat,mask):
      
        mask = mask.float()
      
        self.batch_size, self.seq_length, self.hidden_dim = batch_feat.size()
        
        alphas = batch_feat[:, 0,:]
        
        history = []
        
        for i in range(1, self.seq_length):
             
          
            alphas = alphas.unsqueeze(2)


            e_scores = batch_feat[:, i, :].unsqueeze(1)

            t_scores = self.transitions[:, :].unsqueeze(0)

            scores = e_scores + t_scores + alphas

            new_alphas,idx = torch.max(scores, dim=1)
            
            alphas = alphas.squeeze()

            mask_t = mask[:, i].unsqueeze(-1)
            
            alphas = mask_t * new_alphas + (1 - mask_t) * alphas
            
            history.append(idx)

        best_tags_list = []

        for i in range(self.batch_size):
            end_blank = True
            _, best_last_tag = alphas[i].max(dim=0)
            best_tags = [best_last_tag.item()]

            for p,hist in enumerate(reversed(history)): #use end_blank  to find the last valid pos, if directly use the max in a unvalid pos will cause a bug
                if mask[i,self.seq_length - 1 - p] == 1.0:
                  end_blank = False
                if end_blank:
                  best_tags.append(best_tags[-1])
                else:
                  best_last_tag = hist[i][best_tags[-1]]
                  best_tags.append(best_last_tag.item())

            # Reverse the order because we start from the last timestep
            best_tags.reverse()
            best_tags_list.append(best_tags)

        return torch.LongTensor(best_tags_list).to(batch_feat.device)
        
        
      


In [98]:
crf = CRF(pos2idx["[START]"],pos2idx["[STOP]"])
crf = crf.cuda()
from torch.nn.utils.rnn import pad_sequence
sub_batch = 4
acc_time = 4
acc_counter = 0
EPOCHS = 10



loss_fn = nn.CrossEntropyLoss(ignore_index = 0)#weight = torch.Tensor([1.0,1.0,5.0]))
loss_fn.cuda()
bert = BertModel.from_pretrained('bert-base-uncased')
bert =  bert.cuda()
bert.train()
linear = nn.Linear(768,49).cuda()

opt = torch.optim.Adam(list(bert.parameters()) +list(crf.parameters()) + list(linear.parameters()), lr=2e-5)
for e in range(EPOCHS):
  for b in range(0,len(train_data),sub_batch):
    start = b
    end = min(len(train_data),b+sub_batch)

    tr_X = train_indexed_tokens[start:end]
    tr_Y = train_labels_idx[start:end]
    tr_X_padded = pad_sequence([torch.Tensor(v) for v in tr_X],batch_first = True).cuda().long()
    tr_Y_padded = pad_sequence([torch.Tensor(v) for v in tr_Y],batch_first = True).cuda().long()
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    attention_mask = (tr_X_padded != 0).cuda().long()
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    pos = linear(encoded_bert)

    loss = crf(pos,tr_Y_padded).mean()

    loss.backward()
    acc_counter = acc_counter+1
    if acc_counter == acc_time:
      opt.step()
      acc_counter = 0
      opt.zero_grad()
    if b%100 == 0:
      print ("Epoch =",e,"loss = ",loss.item())

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpqrb_74uj
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Epoch = 0 loss =  97.3095703125
Epoch = 0 loss =  98.65769958496094
Epoch = 0 loss =  79.9422836303711
Epoch = 0 loss =  70.08186340332031
Epoch = 0 loss =  45.1622314453125
Epoch = 0 loss =  58.75858688354492
Epoch = 0 loss =  70.2684326171875
Epoch = 0 loss =  40.951026916503906
Epoch = 0 loss =  28.378543853759766
Epoch = 0 loss =  34.17486572265625
Epoch = 0 loss =  31.145313262939453
Epoch = 0 loss =  37.14376449584961
Epoch = 0 loss =  21.612152099609375
Epoch = 0 loss =  21.580501556396484
Epoch = 0 loss =  19.03482437133789
Epoch = 0 loss =  8.3436918258667
Epoch = 0 loss =  26.33484649658203
Epoch = 0 loss =  7.416454315185547
Epoch = 0 loss =  12.932663917541504
Epoch = 0 loss =  10.651195526123047
Epoch = 0 loss =  9.932064056396484
Epoch = 0 loss =  5.890986442565918
Epoch = 0 loss =  23.297481536865234
Epoch = 0 loss =  6.916168212890625
Epoch = 0 loss =  7.428464889526367
Epoch = 0 loss =  5.253246307373047
Epoch = 0 loss =  5.98457145690918
Epoch = 0 loss =  8.0363731384

In [99]:
valid_trX = pad_sequence([torch.Tensor(v) for v in valid_indexed_tokens],batch_first = True).cuda().long()
valid_trY = pad_sequence([torch.Tensor(v) for v in valid_labels_idx],batch_first = True).cuda().long()

with torch.no_grad():
    bert.eval()
    segments_tensors = torch.zeros(valid_trX.size()).cuda().long()
    attention_mask = (valid_trX != 0).cuda().long()
    encoded_bert, _ = bert(valid_trX, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pos = linear(encoded_bert)
    valid_trY = valid_trY
    pred_trY = crf.decode(pos,(valid_trY!=0).float())
    
print ("Acc = ",(valid_trY[valid_trY!=0] == pred_trY[valid_trY!=0]).float().mean())

Acc =  tensor(0.9799, device='cuda:0')


In [100]:
pd.DataFrame(pred_trY.view(9,-1).cpu().numpy()).applymap(lambda x: idx2pos[x]).loc[1],pd.DataFrame(valid_trY.view(9,-1).cpu().numpy()).applymap(lambda x: idx2pos[x]).loc[1]

(0     [START]
 1          ''
 2          ''
 3          DT
 4          CD
 5         NNS
 6         VBP
 7          JJ
 8         NNS
 9          IN
 10         DT
 11         JJ
 12         NN
 13          :
 14          :
 15        NNP
 16        NNP
 17        NNP
 18        NNP
 19        VBZ
 20         DT
 21         JJ
 22         CC
 23        NNP
 24        NNP
 25        NNP
 26        VBZ
 27         DT
 28         NN
 29          .
 30     [STOP]
 31     [STOP]
 32     [STOP]
 33     [STOP]
 34     [STOP]
 35     [STOP]
 36     [STOP]
 37     [STOP]
 38     [STOP]
 39     [STOP]
 40     [STOP]
 41     [STOP]
 42     [STOP]
 43     [STOP]
 44     [STOP]
 45     [STOP]
 46     [STOP]
 47     [STOP]
 48     [STOP]
 Name: 1, dtype: object, 0     [START]
 1          ''
 2           O
 3          DT
 4          CD
 5         NNS
 6         VBP
 7          JJ
 8         NNS
 9          IN
 10         DT
 11         JJ
 12         NN
 13          :
 14          O
 15        NNP
 

In [81]:
valid_trY[valid_trY!=0] == pred_trY[valid_trY!=0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8)

In [80]:
pred_trY[valid_trY!=0]

tensor([ 3, 45, 31, 10, 32, 16, 30, 12,  1, 31, 35, 22, 36, 36, 36, 28, 36, 30,
        36,  9, 30, 37, 22, 31, 32, 35, 22, 31, 35,  7, 47, 31, 32, 35,  1, 19,
        31, 32, 35, 40, 17,  3, 42, 31, 10, 16, 44, 32, 16, 22, 31, 32, 35, 20,
        36, 36, 47, 31, 32, 45, 36, 36, 47, 31, 32, 40, 17,  3, 31, 10, 16, 37,
        31, 36, 47, 31, 35,  1, 19, 32, 16, 30, 14, 32, 16, 40, 17,  3, 14, 30,
        38, 37, 31, 32, 16, 35, 37, 10, 16, 22, 37, 32, 32, 16, 30, 45, 15, 37,
        31, 35, 22, 31, 32, 35, 40, 17,  3, 38, 14, 37, 22, 33, 22, 31, 35, 16,
        44, 13, 22, 32, 35, 35, 22, 31, 32, 35, 30, 45,  7, 31, 44, 13, 31, 35,
        22, 32, 35, 35, 47, 22, 31, 35, 42, 22, 31, 35, 40, 17,  3, 36, 36, 37,
        22, 31, 32, 35, 22, 38, 47, 35,  1, 19, 31, 35, 32, 35, 35, 30, 45, 22,
        38, 29, 19, 31,  7, 32, 35, 22, 35, 42, 22, 36, 36,  1, 19, 31, 35, 22,
        32, 35, 40, 17,  3, 36, 41, 36, 37, 38, 37, 31, 32, 35,  1, 19, 10, 35,
        16,  1, 35, 36, 36, 22, 36, 40, 

In [101]:
for i in range(9):
  for j in range(49):
    if valid_trY[i,j] != 0:
      if valid_trY[i,j] != pred_trY[i,j]:
        print (i,j)
        print (pred_trY[i,j],valid_trY[i,j])
  

3 9
tensor(37, device='cuda:0') tensor(13, device='cuda:0')
3 12
tensor(14, device='cuda:0') tensor(22, device='cuda:0')
4 20
tensor(32, device='cuda:0') tensor(36, device='cuda:0')
4 21
tensor(35, device='cuda:0') tensor(36, device='cuda:0')
4 33
tensor(35, device='cuda:0') tensor(32, device='cuda:0')


In [91]:
valid_trY[1,28:]

tensor([35, 40, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0], device='cuda:0')

In [92]:
pred_trY[1,28:]

tensor([32, 40, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17], device='cuda:0')